Adversial Attack Simulator

In [4]:
# installing libraries

!pip install datasets
!pip install pandas
!pip install transformers
!pip install torch
!pip install transformers[torch]


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
import re
import html
from collections import Counter

c:\Users\mustafa amanullah\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load the imdb dataset

ds_movie_reviews = load_dataset("stanfordnlp/imdb")

In [34]:
# convert to pandas dataframe

df_movie_reviews = pd.DataFrame(ds_movie_reviews["train"])

In [ ]:
# exploring the dataset

df_movie_reviews.info()
df_movie_reviews.describe()
df_movie_reviews["label"].value_counts()
df_movie_reviews = df_movie_reviews.drop_duplicates(subset=["text"])
df_movie_reviews.duplicated(subset=["text"]).sum()

<class 'pandas.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    25000 non-null  str  
 1   label   25000 non-null  int64
dtypes: int64(1), str(1)
memory usage: 31.4 MB


np.int64(0)

In [ ]:
# initial data cleaning 

def initial_clean_text_movie_reviews(text):
    if not isinstance(text, str):
        return ""
    
    text = html.unescape(text)
    text = re.sub(r"<[^>]*>", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\\[nrt]", " ", text)
    text = text.strip()

    text = re.sub(r"STAR RATING:*?", "", text)
    text = re.sub(r"^\*+\s*", "", text)
    text = re.sub(r"^Saturday Night Friday Night\s*", "", text)
    text = re.sub(r"^s Just Misses the Mark That Littl\s*", "", text)
    return text

df_movie_reviews_prep = df_movie_reviews
df_movie_reviews_prep["text"] = df_movie_reviews["text"].apply(initial_clean_text_movie_reviews)
df_movie_reviews_prep = df_movie_reviews_prep.drop(df_movie_reviews_prep[df_movie_reviews_prep["text"] == ""].index)

In [80]:
pd.set_option('display.max_rows', None)

patterns = df_movie_reviews_prep[df_movie_reviews_prep["text"].str.contains("")]

for i in range (10, 100):
  truncated = patterns["text"].str[:i]
  common_patterns = truncated.value_counts()

  print(common_patterns.head(10))

text
This movie    921
This film     399
This is a     358
I saw this    268
This is on    202
This is th    154
I watched     136
The movie     111
There are      96
This is an     92
Name: count, dtype: int64
text
This movie     908
I saw this     268
This is one    196
This film i    178
This is the    154
I watched t    109
One of the      90
This is an      83
I remember      80
First of al     78
Name: count, dtype: int64
text
This movie i    411
This movie w    197
This is one     196
This film is    177
This is the     154
I saw this m    123
I watched th    108
First of all     78
I saw this f     77
This movie h     74
Name: count, dtype: int64
text
This movie is    407
This movie wa    187
This is one o    178
This film is     174
I saw this mo    122
I watched thi     87
I saw this fi     74
This movie ha     74
I don't know      61
This film was     60
Name: count, dtype: int64
text
This movie is     402
This movie was    186
This is one of    178
I saw this mov    122
I w

In [89]:
df_movie_reviews_prep.info()

<class 'pandas.DataFrame'>
Index: 24902 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    24902 non-null  str  
 1   label   24902 non-null  int64
dtypes: int64(1), str(1)
memory usage: 31.5 MB


In [6]:
# re-exploring the datasets

print(df_train["text"][0] + "\n" + df_test["text"][24])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself. The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot l

In [ ]:
# encode the "text" data for train and test datasets



tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(df_train["text"].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(df_test["text"].tolist(), truncation=True, padding=True, max_length=512)

In [9]:
# 

train_labels = df_train["label"].tolist()
test_labels = df_test["label"].tolist()

set(train_labels), set(test_labels)

({0, 1}, {0, 1})

In [11]:
# create a dataset class for pytorch

import torch

class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
    
train_dataset = IMDBDataset(train_encodings, train_labels)
test_dataset = IMDBDataset(test_encodings, test_labels)

In [12]:
# load the model 

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Loading weights: 100%|██████████| 199/199 [00:01<00:00, 151.77it/s, Materializing param=bert.pooler.dense.weight]                               
BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those pa

In [14]:
# training parameters

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [ ]:
# training the model 

trainer.train()

c:\Users\mustafa amanullah\AppData\Local\Programs\Python\Python314\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Epoch,Training Loss,Validation Loss


In [1]:
# pull the ag news dataset

from datasets import load_dataset

dataset = load_dataset("fancyzhx/ag_news")

print(dataset)

c:\Users\mustafa amanullah\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [34]:
# convert to pandas dataframe

import pandas as pd 

df_news = pd.DataFrame(dataset["train"])

In [39]:
print(df_news["text"][79])

Group to Propose New High-Speed Wireless Format A group of technology companiesincluding Texas Instruments Inc. (TXN.N), STMicroelectronics(STM.PA) and Broadcom Corp. (BRCM.O), on Thursday said theywill propose a new wireless networking standard up to 10 timesthe speed of the current generation.


In [49]:
# pull toxic comments dataset 

from datasets import load_dataset

ds = load_dataset("anitamaxvim/jigsaw-toxic-comments")
print(ds)

c:\Users\mustafa amanullah\AppData\Local\Programs\Python\Python314\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mustafa amanullah\.cache\huggingface\hub\datasets--anitamaxvim--jigsaw-toxic-comments. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating balanced_train split: 100%|██████████| 5408

DatasetDict({
    train: Dataset({
        features: ['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 159571
    })
    test: Dataset({
        features: ['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 63978
    })
    balanced_train: Dataset({
        features: ['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 54083
    })
})


In [50]:
import pandas as pd

ds = pd.DataFrame(ds["train"])

In [53]:
ds.head(10)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,alignment on this subject and which are contra...,0,0,0,0,0,0


In [57]:
ds.isna().sum()
ds.dropna(inplace=True)

In [66]:
from collections import Counter

texts = ds["comment_text"].dropna().astype(str)

first_20_segments = []

for text in texts:
    first_20 = " ".join(text.split()[:43])
    first_20_segments.append(first_20)

counts = Counter(first_20_segments)

print(counts.most_common(10))


[('Welcome! Hello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful: The five pillars of Wikipedia How to edit a page Help pages', 82), ('Thank you for experimenting with Wikipedia. Your test worked, and it has been reverted or removed. Please use the sandbox for any other tests you may want to do. Take a look at the welcome page to learn more about contributing to our', 43), ('"Welcome! Hello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful: The five pillars of Wikipedia Tutorial How to edit a page How', 41), ('" Welcome! Hello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful: The five pillars of Wikipedia How to edit a page Help', 37), ('"Thank you for experimenting 

In [91]:
import html, re

def clean_text_general(text, n=100):
    if not isinstance(text, str):
        return ""
    
    # 1. Unescape HTML entities
    text = html.unescape(text)
    text = html.unescape(text)  # double-unescape if needed
    
    # 2. Remove HTML tags
    text = re.sub(r"<[^>]*>", " ", text)
    
    # 3. Remove stray backslashes
    text = text.replace("\\", "")
    text = text.replace("\" ", "")
    text = text.replace(" \"", "")
    
    # 4. Collapse multiple spaces
    text = re.sub(r"\s+", " ", text)
    
    # 5. Remove source prefix within first n characters
    prefix = text[:n]
    rest = text[n:]
    
    # Match: "(anything) same_text [-—:]"
    prefix_clean = re.sub(r"\((.*?)\)\s*\1\s*[-—:]\s*", "", prefix, flags=re.I)
    
    text = (prefix_clean + rest).strip()
    
    return text

ds["comment_text"] = ds["comment_text"].apply(clean_text_general)

for i in range(20):
    print(ds.comment_text[i+600])

I'm also a sock puppet of this account...SUPRISE!! -sincerely, The man that will track you down from the Internet and kill you
All the sources needed are provided, all you need to do is read the article...but I guess you can't read. Once again you prove you are not even reading the article. Every edit is backed up with the source it came from. I am NOT going to provide you with any details, because you are merely a civillain editor and not a Wikipedia staff member. You have all the info you need within the PIRA text edits. If a actual Wikipedia staff member wants more specifics I will be happy to provide them Look, you are obviously a Anti-IRA hack who is making things up to support your bias. I am neither pro not anti IRA. I have no bias, I merely report facts. I will be checking this page daily, and will continue to submit my PIRA entry because it is by far the most accurate and factually supported. We can do this several times a day if you wish, or you can accept the fact that mine 